In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep
from sklearn import tree, svm, ls

from pandas_datareader import data as pdr
import pandas_ta as ta #commented out for me for now, I (Ben) had some dependency issues

import yfinance as yf
# import quandl as qd # not used, ignor ples
from backtesting import Backtest, Strategy

C:\Users\Staro\.virtualenvs\dsc190-FVH1D0fH\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
df = pd.read_csv('sp500_daily.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600.0,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.594968,601904800.0,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.490066,552160000.0,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.478067,477131200.0,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.521136,447610800.0,AAPL


In [18]:
from itertools import chain
def get_daily_j(id=-1):
    jtimes = pd.DataFrame(list(chain.from_iterable(pd.date_range(timedf["j_start"],timedf["midpoint"]) for _,timedf in timedf.iterrows())), columns=("date",))
    jtimes['date'] = jtimes['date'].astype('datetime64[ns]')
    jtimes['period'] = (jtimes['date'].isin(timedf['midpoint'])).shift(1).cumsum().fillna(0).astype(int)
    df_ = df.copy()
    if 'date' not in df_.columns: 
        df_['date'] = pd.to_datetime(df_['Date'])
    df_['date'] = pd.to_datetime(df_['date'])
    df_ = df_.merge(jtimes, how='inner', left_on='date', right_on='date')
    if id==-1: 
        return df_ # this filters out periods that dont end prettily
    else:
        return df_[df_['period']==id].drop(columns='period')
# get_j_df = get_daily_j
def get_daily_k(id=-1):
    ktimes = pd.DataFrame(list(chain.from_iterable(pd.date_range(timedf["midpoint"],timedf["k_end"]) for _,timedf in timedf.iterrows())), columns=("date",))
    ktimes['date'] = ktimes['date'].astype('datetime64[ns]')
    ktimes['period'] = (ktimes['date'].isin(timedf['k_end'])).shift(1).cumsum().fillna(0).astype(int)
    df_ = df.copy()
    if 'date' not in df_.columns: 
        df_['date'] = pd.to_datetime(df_['Date'])
    df_['date'] = pd.to_datetime(df_['date'])
    df_ = df_.merge(ktimes, how='inner', left_on='date', right_on='date')
    if id==-1: 
        return df_ # this filters out periods that dont end prettily
    else:
        return df_[df_['period']==id].drop(columns='period')
get_daily_j()#.period.plot()
def get_cum_return(data):
    '''gets cumulative return based on adjusted closing price of all tickers in input dataframe'''
    df_ = data.set_index('Name')

    # df_['pct_change'] = df_['Adj Close'].groupby('Name').pct_change()
    df_['pct_change'] = df_['Adj Close'].groupby('Name').pct_change()
    df_['cum_return'] = (df_['pct_change']+1).groupby('Name').cumprod().fillna(1)-1
    df_['adj_close_shifted'] = df_['Adj Close'].groupby('Name').shift(3)#.bfill(0) # filing the value with the backfill TODO check this later
    df_['adj_change'] = df_['Adj Close']/df_['adj_close_shifted'] #.bfill(0) 
    df_['adj_close_shifted'] = df_['Adj Close'].groupby('Name').shift(3)#.bfill(0) # filing the value with the backfill TODO check this later
    df_['adj_change'] = df_['Adj Close']/df_['adj_close_shifted'] #.bfill(0) 
    
    return df_

def get_percents(n): 
    '''this function gets us n percent number of tickers'''
    return features.shape[0]//n+1

def get_portfolios(period=0, n=50):   
    '''returns the portfolios we want from a particular period'''
    ## Gets the portfolio(s) in question we want to look at
    data = get_cum_return(get_daily_j(period))
    features = data.groupby('Name').tail(1).sort_values(['cum_return'], ascending=False) # sorts tickers by cumulative return

    winner_tickers = features[:get_percents(n)].index # we get the top/bottom n percent tickers
    loser_tickers = features[-get_percents(n):].index # only winner and loser portfolios for now, we could expand later
    # TODO add more portfolios if we have time
    # print(len(winner_tickers), len(loser_tickers))
    print('.', end='')
    return (winner_tickers.to_list(), loser_tickers.to_list())

def get_portfolio_performance(period=0, n=2, weights=None, hold=False):
    '''Gets all portfolios' cumulative return performance based on n period 
    keyword args:
    period      -- the nth period of data we are looking at, default 0
    portfolio   -- the particular type of portfolio we want to be looking at (winner or loser, etc) default winner/momentum
    n           -- the percentage of tickers we want to be looking at
    weights     -- how to weight the portfolio values. if unspecified (None) we assume equal weighting in the portfolio
    hold        -- if true returns the performance evaluation of the holding period k, else returns the performance of the observation period j
    '''
    winner_tickers, loser_tickers = get_portfolios(period, n)

    ## evaluates the performance of portfolios on either hold or observational data
    eval_df = get_cum_return(get_k_df(period)) if hold else get_cum_return(get_daily_j(period))# we get the return from the hold period
    
    weights = np.ones(get_percents(n))/(get_percents(n)) if not weights else weights # set weights
    print('weights', len(weights))
    winner_eval = eval_df.loc[winner_tickers] # first the winners
    winner_eval = winner_eval.pivot_table(index='Date',columns='Name')['pct_change'].fillna(0)
    winner_performance = (winner_eval.dot(weights)+1).cumprod() - 1 # cumulative return

    loser_eval = eval_df.loc[loser_tickers] # then the losers
    loser_eval = loser_eval.pivot_table(index='Date',columns='Name')['pct_change'].fillna(0)
    loser_performance = (loser_eval.dot(weights)+1).cumprod() - 1 # cumulative return

    output = pd.DataFrame({'winners': winner_performance, 'losers': loser_performance})
    return output
get_portfolio_performance()
def generate_daily_timetable(j, k):
    '''aims to generate the MONTHLY table of times for each observe/hold period. The midpoint in this context is where we switch from the observation period len=j to the holding period len=k'''
    j_end = pd.date_range(pd.to_datetime(df.Date.min())+pd.DateOffset(days=j), pd.to_datetime(df.Date.max()), freq=str(k)+"D") # the 'midpoints' for each observe and hold period. 
    j_start = j_end + pd.DateOffset(days=-j) # based on the midpoint, get the start point for each observe and hold period
    k_end = j_end + pd.DateOffset(days=k-1) # based on the midpoint, get the end point for each observe and hold period
    timetable = pd.DataFrame({'j_start': j_start, 'midpoint': j_end, 'k_end': k_end}).infer_objects()
    return timetable, j, k

NameError: name 'features' is not defined

In [8]:
# get features
data = pd.read_csv('daily_j10_k10_features.csv')
data

,index,adj_close_mean,adj_close_min,adj_close_max,adj_close_std,adj_close_diff_mean,adj_close_diff_min,adj_close_diff_max,adj_close_diff_std,pct_chng_mean,...,SMA_5_std,log_ret_mean,log_ret_min,log_ret_max,log_ret_std,cumprod_last,mom_cumprod_last,rev_cumprod_last,log_ret_last,label
0,0,1136.777327,1091.760010,1150.229980,15.081346,-2.944999,-24.719971,14.199951,11.504507,-0.002423,...,4.731755,-0.002471,-0.022390,0.012422,0.009865,-0.036390,0.243343,-0.159836,-0.022390,1.0
1,1,1091.051426,1073.869995,1103.319946,9.724233,1.089986,-12.969971,15.319946,12.318654,0.000896,...,0.673109,0.000849,-0.011888,0.014165,0.010345,0.005963,0.007389,0.018856,0.012890,0.0
2,2,1074.803331,1056.739990,1097.280029,14.249722,-2.721252,-34.170044,13.780029,14.728086,-0.002151,...,4.365267,-0.002227,-0.031636,0.012956,0.012831,-0.019840,-0.043279,-0.025423,-0.002748,0.0
3,3,1103.662012,1094.869995,1109.170044,6.191079,3.285004,-1.160034,7.239990,3.558300,0.002393,...,0.000000,0.002386,-0.001046,0.006563,0.003097,0.012001,0.000000,0.000000,-0.001046,0.0
4,4,1108.627767,1094.599976,1122.969971,10.650013,3.546249,-2.300049,11.219971,4.937411,0.002855,...,6.100350,0.002843,-0.002083,0.010107,0.004313,0.025918,-0.021017,-0.008854,0.003729,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,402,3840.132031,3798.909912,3855.360107,23.651921,14.112549,-11.600098,52.940186,27.898588,0.002972,...,0.000000,0.002950,-0.003015,0.013839,0.006524,0.014860,0.044117,-0.011486,0.003609,1.0
403,403,3805.967801,3714.239990,3871.739990,52.409185,2.764984,-98.850098,59.620117,59.098503,0.000731,...,9.553497,0.000637,-0.026013,0.015924,0.014617,0.005746,-0.017804,-0.004672,0.010795,0.0
404,404,3908.795724,3886.830078,3934.830078,17.095568,8.000000,-4.360107,28.760010,13.012541,0.001759,...,5.915260,0.001753,-0.001114,0.007372,0.003137,0.012349,0.063108,0.054149,0.004700,1.0
405,405,3909.700021,3876.500000,3932.590088,22.990007,-1.193359,-30.209961,44.059814,25.395870,-0.000245,...,5.297419,-0.000260,-0.007763,0.011288,0.005943,-0.001821,0.022329,-0.033275,0.011288,1.0


In [9]:
timedf, j, k = generate_daily_timetable(10,10) # change your values as needed
print(timedf.shape[0])
timedf

408


,j_start,midpoint,k_end
0,2010-01-04,2010-01-14,2010-01-23
1,2010-01-14,2010-01-24,2010-02-02
2,2010-01-24,2010-02-03,2010-02-12
3,2010-02-03,2010-02-13,2010-02-22
4,2010-02-13,2010-02-23,2010-03-04
...,...,...,...
403,2021-01-16,2021-01-26,2021-02-04
404,2021-01-26,2021-02-05,2021-02-14
405,2021-02-05,2021-02-15,2021-02-24
406,2021-02-15,2021-02-25,2021-03-06


# model training

In [13]:
data.iloc[:,-1]

0      1.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
402    1.0
403    0.0
404    1.0
405    1.0
406    0.0
Name: label, Length: 407, dtype: float64

In [16]:
pd.merge(timedf[['midpoint']], data[['label']], left_index=True, right_index=True)

,midpoint,label
0,2010-01-14,1.0
1,2010-01-24,0.0
2,2010-02-03,0.0
3,2010-02-13,0.0
4,2010-02-23,0.0
...,...,...
402,2021-01-16,1.0
403,2021-01-26,0.0
404,2021-02-05,1.0
405,2021-02-15,1.0
